<a href="https://colab.research.google.com/github/Taedriel/ZSL-v2/blob/wordEmbedding/WordsEmbeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# %cd /content/drive/MyDrive/Kingston/ZSL-v2/

In [ ]:
!pip install transformers wget tensorflow_datasets wikipedia deprecated vecto
!mkdir temp

In [13]:
import tensorflow_datasets as tfds
import tensorflow as tf

import wikipedia
import torch
import numpy as np
from transformers import BertTokenizer, BertModel

import logging
logging.basicConfig(level = logging.INFO, filename = "BERT.log" )
# logging.basicConfig(level = logging.INFO)

from tqdm import tqdm
from deprecated import deprecated
from typing import List, Tuple
from os.path import exists


In [26]:
class WordToVector:

    def __init__(self, list_tags : List[str] = []):
        self.list_tags = list_tags
        self.embeddings = []

    def export(self, filename):
        return NotImplementedError

    def convert(self):
        return NotImplementedError


class BERTModel(WordToVector):

    temp_dir = "./temp"

    def __init__(self, list_tag : List[str] = [], big: bool = False):
        super(BERTModel, self).__init__(list_tag)
        self.list_articles = {}

        self.model_size = "bert-large-uncased" if big else "bert-base-uncased"
        self.cosine_sim_matrix = None

        self.tokenizer = BertTokenizer.from_pretrained(self.model_size, padding=True, truncation=True,)
        self.model = BertModel.from_pretrained(self.model_size, output_hidden_states = True)

        self.merging_strategy = Sum4LastLayers()

        self.model.eval()

    def export(self, filename):
        if len(self.embeddings) == 0:
            raise Exception("Tags not converted yet !")
        
        try:
            f = open(filename, "w")
        except OSError:
            raise OSError("Could not open file")

        with f:
            for embedding in self.embeddings:
                line = ",".join(list(map(lambda x: str(float(x)), embedding[1])))
                print(embedding[0], ",", line, sep="", file=f)

    @deprecated(reason = "use constructor after manual import of the file is preffered")
    def import_tag_list(self, filename):
        self.embeddings.clear()
        self.cosine_sim_matrix = None

        try:
            f = open(filename, "r")
        except OSError:
            return OSError("Could not open file")

        with f:
            data = f.read().split("\n")
            for item in data:
                if item not in self.list_tags and str.strip(item) != "":
                    self.list_tags.append(item)
            print(self.list_tags)
            logging.info(f"Import finished : {len(self.list_tags)} elements imported.")

    def _retrieve_summary(self, title, depth : int, include_ambiguous : bool = True) -> Tuple[str, str]:
            if depth == 0:
                logging.warning(f"Maximum recursion depth for {title}")
                return None
            
            filename = BERTModel.temp_dir + "/" + title.replace(" ", "_")
            page = None

            if exists(filename):
                with open(filename, "r") as f:
                    return (title, f.read())

            search_result = wikipedia.search(title, suggestion = False)
            try:
                summary = wikipedia.page(search_result[0]).summary
                with open(filename, "w") as f:
                    logging.info(f"Saving {title}")
                    print(summary, file=f)
                return (search_result[0], summary)

            except wikipedia.PageError as e:
                if search_result[0] is not None:
                    logging.warning(f"{title} missSpelled or article missing. Best find is {search_result[0]}")
                    return self._retrieve_summary(search_result[0], depth - 1, include_ambiguous)  
                else: return None

            except  wikipedia.DisambiguationError as e:
                if include_ambiguous:
                    logging.warning(f"{title} is ambiguous")
                    return self._retrieve_summary(search_result[1], depth - 1, include_ambiguous)
                else: return None
                
            except IndexError as e:
                logging.warning(f"No result for {title}, skipping")
                return None
    

    def _retrieve_all_articles(self, include_ambiguous : bool = False):
        logging.info("Starting retrieveing articles...")
        nb_success = 0

        nb_token = len(self.list_tags)
        for i, tag in tqdm(enumerate(self.list_tags), total=nb_token):
            res = self._retrieve_summary(tag, 5, include_ambiguous)
            self.list_articles[tag] = res
            
            if res is not None: 
                nb_success += 1

        logging.info(f"Finished retrieving {nb_success} article(s) !")


    def convert(self, force_retrieve : bool = False):
        """ convert all word in their embeddings"""
        if len(self.list_tags) == 0:
            raise Exception("not tags yet !")

        if len(self.list_tags) != len(self.list_articles) or force_retrieve:
            self._retrieve_all_articles()

        logging.info("Starting converting tokens...")
        nb_token = len(self.list_tags)
        current_percent = 0.00

        for i, tag in tqdm(enumerate(self.list_tags), total = nb_token):
            
            if self.list_articles[tag] is None: continue

            article_size = len(self.list_articles[tag][1])
            max_size_article = min(article_size, 300)

            tag_plus_context = tag + ". " + self.list_articles[tag][1][:max_size_article]
            inputs = self.tokenizer(tag_plus_context, return_tensors = "pt")

            with torch.no_grad():
                outputs = self.model(**inputs)

            hidden_states = outputs[2]

            # [# layers, # batches, # tokens, # features] ==> [# tokens, # layers, # features]
            token_embeddings = torch.stack(hidden_states, dim=0)
            token_embeddings = torch.squeeze(token_embeddings, dim=1)
            token_embeddings = token_embeddings.permute(1,0,2)

            # apply different strategy to summarize word embeddings
            # tokenized_text = self.tokenizer.tokenize(tag)
            # acc = []
            # for i, token in reversed(list(enumerate(tokenized_text))):

            #     embed = self.merging_strategy.merge(token_embeddings[i+1])
            #     if i == 0:
            #         if len(acc) != 0:
            #             embed = torch.mean(torch.stack([x[1] for x in acc]), dim=0)
            #             token = tag
            #             acc = []

            #         self.embeddings.append((token, embed))
            #     else:
            #         acc.append((token, embed))
            
            # here we are just taking the [CLS] (for classification) as an embedding for the tag
            self.embeddings.append((tag, self.merging_strategy.merge(token_embeddings[0])))


    def compute_sim(self):
        """ compute cosine similarity between all vectors """
        if len(self.embeddings) == 0:
            raise Exception("Tags not converted yet !")

        logging.info("Computing cosine similarity, this could take some time...")

        n_tokens = len(self.embeddings)
        self.cosine_sim_matrix = [[1 for j in range(n_tokens)] for i in range(n_tokens)]

        for j, vector in tqdm(enumerate(self.embeddings), total = len(self.embeddings)):

            for i, other_vector in enumerate(self.embeddings):

                if i == j:
                    continue

                
                cos = torch.nn.CosineSimilarity(dim=0)
                similarity = cos(vector[1], other_vector[1])

                self.cosine_sim_matrix[i][j] = similarity
                self.cosine_sim_matrix[j][i] = similarity

    def export_sim_matrix(self, filename):
        if self.cosine_sim_matrix == None:
            self.compute_sim()
        
        try:
            f = open(filename, "w")
        except OSError:
            raise OSError("Could not open file")

        with f:
            print("/", ",".join([tag[0] for tag in self.embeddings]), sep = ",", file = f)

            for j, tag_y in enumerate(self.embeddings):
                print(tag_y[0], ",".join( [str(round(float(self.cosine_sim_matrix[j][i]), 3)) for i in range(len(self.embeddings))]), sep = ",", file = f)

    def sim_between(self, token1, token2):
        if self.cosine_sim_matrix is None:
            self.compute_co_sim()

        index1 = [i for i, v in enumerate(self.embeddings) if v[0] == token1][0]
        index2 = [i for i, v in enumerate(self.embeddings) if v[0] == token2][0]

        return self.cosine_sim_matrix[index1][index2]

    def get_embedding_of(self, token):
        res = [v for v in self.embeddings if v[0] == token]
        if len(res) == 0:
            raise Exception("no such token")
        
        return res[0]

    def get_nearest_embedding_of(self, embedding, nb = 10):

        if nb > len(self.embeddings):
            raise Exception("nb too high, not enough token")

        nearest = []
        for e in self.embeddings:

            cos = torch.nn.CosineSimilarity(dim=0)
            similarity = cos(embedding, e[1])

            nearest.append((e[0], similarity))
        
        nearest.sort(key = lambda tup : tup[1])
        return nearest[-nb:]

    def get_class_list(self):
        return [x[0] for x in self.embeddings]

class Sum4LastLayers:

    def merge(self, vector):
        return torch.sum(vector[-4:], dim = 0)





In [4]:
class EmbeddingTest:

    def __init__(self, filename : str):

        self.file = filename
        self.embeddings = {}

        self._load_file()

    def _load_file(self):
        try:
            with open(self.filename, "r") as f:
                lines = f.readlines()
                
            for line in lines:
                self.embeddings[line[0]] = line[1:]

        except IOError as e:
            raise IOError(f"No file {self.filename}")

    def evaluate(self):
        raise NotImplementedError

In [5]:
labels_path = tf.keras.utils.get_file('ImageNetLabels.txt','https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt')
imagenet_labels = np.array(open(labels_path).read().splitlines())
imagenet_labels = list(imagenet_labels)
len(imagenet_labels)

1001

In [6]:
sub_imagenet_labels = imagenet_labels[:200]
print(len(sub_imagenet_labels))

200


In [7]:
wikipedia.set_rate_limiting(True)

In [9]:
model = BERTModel(sub_imagenet_labels, big = True)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# model.import_tag_list("en-basic")
model.convert()
model.export("imageNet-embeddeding.csv")
print(len(model.get_class_list()))

# model.computeCoSim()
# model.export_sim_matrix("sim_matrix.csv")

In [24]:
#@title Article to search for { run: "auto", vertical-output: true, display-mode: "both" }
totest = "queen" #@param {type:"string"}

result = wikipedia.search(totest, suggestion = False)
print(result)
print(f"first result with suggestion is: {result[0]}")

try:
    print(wikipedia.page(result[0], auto_suggest=False, redirect=True))
    print(wikipedia.page(totest, auto_suggest=False, redirect=True))
except Exception as e:
    print(e)

['Queen', 'Queen (band)', 'Queen Elizabeth The Queen Mother', 'Queen Victoria', 'Queen (Queen album)', 'The Queen', 'Mary, Queen of Scots', 'Elizabeth II', 'Elizabeth I', 'Queen Mary']
first result with suggestion is: Queen
"Queen" may refer to: 
Queen regnant
List of queens regnant
Queen consort
Queen dowager
Queen mother
Queen (Marvel Comics)
Evil Queen
Red Queen (Through the Looking-Glass)
Queen of Hearts (Alice's Adventures in Wonderland)
Queen (chess)
Queen (playing card)
Carrom
Queen (band)
Queen (Queen album)
Queen (Kaya album)
Queen (Nicki Minaj album)
Ten Walls
Lovers Rock
G Flip
R.O.S.E.
Stoner Witch
Too Bright
Shawn Mendes
Deltarune Chapter 2 OST
Record
Q.U.E.E.N.
Queen Records
Queen (magazine)
Queen: The Story of an American Family
Alex Haley's Queen
Queen (2014 film)
Queen (2018 film)
Queen (web series)
Queen, New Mexico
Queen, Pennsylvania
May Queen
Queen of Heaven
Queen of heaven (antiquity)
eusociality
Queen ant
Queen bee
Queen (butterfly)
cat
Queen (Canadian automobile

/usr/local/lib/python3.7/dist-packages/wikipedia/wikipedia.py:389: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.7/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [25]:
test_model = BERTModel(["King", "Queen regnant", "man", "woman"], big = True)
test_model.convert()

man = test_model.get_embedding_of("man")[1]
woman = test_model.get_embedding_of("woman")[1]
king = test_model.get_embedding_of("King")[1]

totest = king.sub(man).add(woman)
print(test_model.get_nearest_embedding_of(totest, nb=4))

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
  0%|          | 0/4 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/wikipedia/wikipedia.py:389: UserWarning: No parser was explicitly specified, s

KeyError: ignored

In [13]:
# 821 / 1001 classes = 82%
# x
# 1024 col

!ls "/content"

 imageNet-embeddeding.csv			   temp
 sample_data					   vecto
'show?code=XZOn0J7Z8fzFMt7Tw1mGS6uI1SYfCfTyJQTV'


In [11]:
!rm -f /content/temp/*